In [ ]:
!pip install google-cloud-secret-manager

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 3.9 MB/s eta 0:00:00


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.cloud import secretmanager

def access_secret_version(project_id, secret_id, version_id="latest"):
    # Create the Secret Manager client.
    client = secretmanager.SecretManagerServiceClient()

    # Build the resource name of the secret version.
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"

    # Access the secret version.
    response = client.access_secret_version(name=name)

    # Return the decoded payload.
    return response.payload.data.decode('UTF-8')

# replace 'your-secret-name' with the name of the secret you created
my_secret = access_secret_version('final-385413', 'api_key')

In [ ]:
!pip install llama_index

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.5/440.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.4/969.4 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
# set text wrapping
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
from pathlib import Path
from llama_index import SimpleDirectoryReader, StorageContext, LLMPredictor, PromptHelper, GPTVectorStoreIndex, ServiceContext, load_index_from_storage, download_loader
from langchain import OpenAI
import sys
import os
import random

os.environ["OPENAI_API_KEY"] = my_secret

In [ ]:
import nltk

# Only run this once, they will be downloaded.
nltk.download('stopwords',quiet=True)
nltk.download('wordnet',quiet=True)
nltk.download('punkt',quiet=True)
nltk.download('omw-1.4',quiet=True)

True

In [ ]:
!pip install pdf2image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir("/content/drive/MyDrive/Datasets/")

In [ ]:
data = SimpleDirectoryReader('golf').load_data()

In [ ]:
# https://gpt-index.readthedocs.io/en/latest/reference/service_context/prompt_helper.html

# vector
max_input = 4000
tokens = 200
chunk_size = 100
chunk_overlap_ratio = 0.2
#max_chunk_overlap = 20

promptHelper = PromptHelper(max_input, tokens, chunk_overlap_ratio=chunk_overlap_ratio, chunk_size_limit=chunk_size)
llmPredictor=LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=tokens))


service_context = ServiceContext.from_defaults(llm_predictor=llmPredictor, prompt_helper=promptHelper)
storage_context = StorageContext.from_defaults()
index = GPTVectorStoreIndex.from_documents(
    documents=data,
    service_context=service_context,
    storage_context=storage_context
)
persist_dir =  f'/content/drive/MyDrive/Datasets/persist/'
storage_context.persist(persist_dir=persist_dir)

In [ ]:
def answerMe(question):
  query_engine = index.as_query_engine()
  response = query_engine.query(question).response
  return response[1:]

In [ ]:
#create greetings and greetings function

GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["Hello"]


# Checking for greetings
def greeting(sentence):
    """If user's input is a greeting, return a greeting response"""
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [ ]:
#Chatbot interaction code

flag=True
print("Welcome to the Mike Malaska Golf Chatbot. To end session please type exit")
print("\n")

while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if user_response!='exit':
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Answer: You are welcome!")
        else:
            if(greeting(user_response)!=None):
                print("Answer: "+greeting(user_response))
            else:
                print("Answer: ",end="")
                print(answerMe(user_response))
                print("\n")
    else:
        flag=False
        print("Thank you for using the Mike Malaska Golf Chatbot. Good bye.")

Welcome to the Mike Malaska Golf Chatbot. To end session please type exit


Answer: Hello
Answer: 
To get more consistent at golf, practice regularly and focus on improving your technique. Make sure to focus on the basics such as your grip, stance, and swing. Additionally, pay attention to the details of your technique, such as keeping your head still, and practice those elements until they become second nature. Aiming is also important, so make sure to use alignment sticks to ensure you are aiming correctly and that your alignment is consistent. Take control of things you have control of, such as your grip, aim, posture, ball position, and club face, and make sure to be consistent with them. Make sure to square the club face correctly by placing your hands on the club correctly and sliding them apart. Pay special attention to your hands and how they control the club face, and practice until you can make the club face work regardless of your swing. When practicing, start on the putting